<a href="https://colab.research.google.com/github/RC11-SkillsClass2022-23/Dongchen-Du/blob/main/blur_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import numpy as np
from os.path import dirname, join

In [ ]:

# https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
prototxt_path = join(dirname('/content/drive/MyDrive/RC11_2022-23/pretrainedmodel'), "/content/drive/MyDrive/RC11_2022-23/pretrainedmodel/deploy.prototxt")
# https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20180205_fp16/res10_300x300_ssd_iter_140000_fp16.caffemodel
model_path = join(dirname('/content/drive/MyDrive/RC11_2022-23/pretrainedmodel'), "/content/drive/MyDrive/RC11_2022-23/pretrainedmodel/res10_300x300_ssd_iter_140000_fp16.caffemodel")
# load Caffe model
model = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

In [ ]:
# read the desired image
image = cv2.imread("/content/drive/MyDrive/RC11_2022-23/videotoproress/5adc73bc015d4b7047f8de014d179182.jpg")
# get width and height of the image
h, w = image.shape[:2]
# gaussian blur kernel size depends on width and height of original image
kernel_width = (w // 7) | 1
kernel_height = (h // 7) | 1
# preprocess the image: resize and performs mean subtraction
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
# set the image into the input of the neural network
model.setInput(blob)
# perform inference and get the result
output = np.squeeze(model.forward())


In [ ]:
for i in range(0, output.shape[0]):
    confidence = output[i, 2]
    # get the confidence
    # if confidence is above 40%, then blur the bounding box (face)
    if confidence > 0.4:
        # get the surrounding box cordinates and upscale them to original image
        box = output[i, 3:7] * np.array([w, h, w, h])
        # convert to integers
        start_x, start_y, end_x, end_y = box.astype(int)
        # get the face image
        face = image[start_y: end_y, start_x: end_x]
        # apply gaussian blur to this face
        face = cv2.GaussianBlur(face, (kernel_width, kernel_height), 0)
        # put the blurred face into the original image
        image[start_y: end_y, start_x: end_x] = face
        cv2.imwrite('/content/drive/MyDrive/RC11_2022-23/video_done/blurred-face.jpg', image)